In [1]:
import Parser
import Process_Data
import Feature_Engineering as fe

import datetime
import pandas as pd
from selenium import webdriver
import os


## DOWNLOAD DATA

In [10]:
## PARSE
matches = []
hrefs = set()

script_dir = os.path.dirname(os.path.realpath('chromedriver'))
webdriver_path = os.path.join(script_dir, "chromedriver")

startdate = datetime.date(2021, 5, 22)
endtime = datetime.date(2021, 5, 23)
Parser.parse(startdate, enddate, matches, hrefs, webdriver_path)


## UPDATE OLD MATCHES
filename = 'Data/old_matches.csv'
old_matches = Process_Data.csv_to_list(filename)

matches = Process_Data.unpack_and_change(matches)
Process_Data.make_csv(matches, 'Data/new_matches.csv')
matches_after_parse = Process_Data.csv_to_list('Data/new_matches.csv')
Process_Data.make_csv(old_matches + matches_after_parse, 'Data/old_matches.csv')


## PROCESS OLD MATCHES
Process_Data.delete_trash(old_matches)
Process_Data.split_sets_and_dur(old_matches)
Process_Data.fix_set_score(old_matches)
Process_Data.split_stats(old_matches)
Process_Data.create_srv_games(old_matches)
Process_Data.create_bp_stat(old_matches)


## PROCESS NEW MATCHES
filename = 'Data/new_matches.csv'
new_matches = Process_Data.csv_to_list(filename)
Process_Data.delete_trash(new_matches)
Process_Data.split_sets_and_dur(new_matches)
Process_Data.fix_set_score(new_matches)
Process_Data.split_stats(new_matches)
Process_Data.create_srv_games(new_matches)
Process_Data.create_bp_stat(new_matches)

## UNION MATCHES --> CSV
matches = old_matches + new_matches
filename = 'Data/matches_final.csv'
Process_Data.to_csv(matches, filename)

## CSV --> PICKLE
data = pd.read_csv(filename)
filename = 'Data/matches_final.pkl'
data.to_pickle(filename)


Parsed matches on date: 2020-11-18 4


/Users/yasha_ten/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## FEATURE ENGINEERING

In [2]:
## READ DATA
df = pd.read_pickle('test.pkl') 
players1 = pd.read_pickle('Data/players.pkl')
players2 = pd.read_pickle('Data/players.pkl')

## CHANGE TIME TYPE
fe.fix_time(df)

## fix players' names for transporting stats
fe.fix_names1(players1)
fe.fix_names2(players2)
players1 = players1.set_index(['name'])
players2 = players2.set_index(['name'])

## transporting stats players --> dataframe
fe.add_stats(df, players1)
fe.add_stats(df, players2, 0)


## fix letters in data
fe.fix_letters(df)

## create retirement stat
fe.create_retirement_stat(df)


## converting age to datetime
fe.fix_age(df, 1)
fe.fix_age(df, 2)



## converting some stats to datetime in df for the next steps
df['date'] = pd.to_datetime(df['date'])
df['p1_birth'] = pd.to_datetime(df['p1_birth'])
df['p2_birth'] = pd.to_datetime(df['p2_birth'])

# count prc of winning points on return
fe.create_feature_WRP(df)

## normalize to per_game stats
fe.create_aces_per_game(df)
fe.create_double_faults_per_game(df)


## create stats based on common opponents
stats = ['first_serve_prc_match_player1', 'first_serve_points_prc_match_player1',
         'second_serve_points_prc_match_player1', 'winning_on_return_prc_player1',
        'aces_per_game_player1', 'df_per_game_player1', 'break_points_prc_match_player1']
fe.create_common_stats(df, stats)


## create new features
fe.create_feature_WSP(df)
fe.create_feature_COMPLETE(df)
fe.create_feature_SERVEADV(df)

# add invert matches
df = fe.invert(df)

## drop Nan stats
df.dropna(subset=['p1_height'], inplace=True)
df.dropna(subset=['p2_height'], inplace=True)
df.dropna(subset=['p1_birth'], inplace=True)
df.dropna(subset=['p2_birth'], inplace=True)
df.dropna(subset=['k1'], inplace=True)
df.dropna(subset=['k2'], inplace=True)

# FIX SOME STATS' FORMAT
df = fe.fix_format(df)

# CREATE LAG FEATURES USING MAIN STATS
df = fe.build_lag_features(df)


df.rename(columns={"player1": "p1", "player2": "p2", "player1_win": "p1_win"}, inplace=True)
df = df.set_index(["date", "p1", "p2"])

df.head(3)

FIXING PLAYERS' NAMES...


IntProgress(value=0, max=1357)

IntProgress(value=0, max=1357)

PLAYERS' NAMES FIXED

IMPORTING STATIC STATS...


IntProgress(value=0, max=2)

...

IntProgress(value=0, max=2)

STAT df_per_game_common CREATED
CREATING STAT break_points_prc_match__common...


IntProgress(value=0, max=2)

STAT break_points_prc_match_common CREATED
ADDING INVERT MATCHES...
INVERT MATCHES ADDED

CREATING LAG FEATURES...
LAG FEATURES CREATED



index  \
date       p1             p2                   
2020-11-18 Djokovic N.    Medvedev D.      0   
           Medvedev D.    Djokovic N.      1   
           Schwartzman D. Zverev A.        2   

                                                                               match_url  \
date       p1             p2                                                               
2020-11-18 Djokovic N.    Medvedev D.  https://www.sofascore.com/medvedev-djokovic/HX...   
           Medvedev D.    Djokovic N.  https://www.sofascore.com/medvedev-djokovic/HX...   
           Schwartzman D. Zverev A.    https://www.sofascore.com/zverev-schwartzman/Z...   

                                      status Location  Surface  match_dur  \
date       p1             p2                                                
2020-11-18 Djokovic N.    Medvedev D.  Ended  London       0.0         81   
           Medvedev D.    Djokovic N.  Ended  London       0.0         81   
           Schwartzman D. Zverev A.    Ended  London       0.0        131   

                                         k1    k2  p1_win  round  \
date       p1             p2                                       
2020-11-18 Djokovic N.    Medvedev D.  1.40  3.00       0    0.0   
           Medvedev D.    Djokovic N.  3.00  1.40       1    0.0   
           Schwartzman D. Zverev A.    3.75  1.28       0    0.0   

                                       duration_set1  duration_set2  \
date       p1             p2                                          
2020-11-18 Djokovic N.    Medvedev D.             47             34   
           Medvedev D.    Djokovic N.             47             34   
           Schwartzman D. Zverev A.               39             53   

                                       duration_set3  duration_set4  \
date       p1             p2                                          
2020-11-18 Djokovic N.    Medvedev D.            NaN            NaN   
           Medvedev D.    Djokovic N.            NaN            NaN   
           Schwartzman D. Zverev A.             39.0            NaN   

                                       duration_set5  aces_match_player1  \
date       p1             p2                                               
2020-11-18 Djokovic N.    Medvedev D.            NaN                 3.0   
           Medvedev D.    Djokovic N.            NaN                 9.0   
           Schwartzman D. Zverev A.              NaN                 0.0   

                                       aces_match_player2  \
date       p1             p2                                
2020-11-18 Djokovic N.    Medvedev D.                 9.0   
           Medvedev D.    Djokovic N.                 3.0   
           Schwartzman D. Zverev A.                  10.0   

                                       double_faults_match_player1  \
date       p1             p2                                         
2020-11-18 Djokovic N.    Medvedev D.                          3.0   
           Medvedev D.    Djokovic N.                          4.0   
           Schwartzman D. Zverev A.                            2.0   

                                       double_faults_match_player2  \
date       p1             p2                                         
2020-11-18 Djokovic N.    Medvedev D.                          4.0   
           Medvedev D.    Djokovic N.                          3.0   
           Schwartzman D. Zverev A.                            3.0   

                                       first_serve_prc_match_player1  \
date       p1             p2                                           
2020-11-18 Djokovic N.    Medvedev D.                       0.632353   
           Medvedev D.    Djokovic N.                       0.714286   
           Schwartzman D. Zverev A.                         0.677419   

                                       first_serve_prc_match_player2  \
date       p1             p2                                           
2020

## CREATE MODELS

In [3]:
import Create_Models as cm

In [7]:
df = pd.read_pickle('Data/all_features.pkl')
data = cm.create_data1(df)
y_pred = cm.build_model_classifier(data, startdate='')

In [10]:
df.tail(3)

p2_filter_lag_1_max_points_in_a_row_match__median_7  \
date       p1          p2                                                                 
2020-11-21 Thiem D.    Djokovic N.                                                7.0     
2020-11-22 Medvedev D. Thiem D.                                                   6.0     
           Thiem D.    Medvedev D.                                                7.0     

                                    p1_lag_1_points_won_match__pctl10_28  \
date       p1          p2                                                  
2020-11-21 Thiem D.    Djokovic N.                                  60.2   
2020-11-22 Medvedev D. Thiem D.                                     47.4   
           Thiem D.    Medvedev D.                                  60.2   

                                    p1_lag_1_second_serve_prc_match__pctl90_7  \
date       p1          p2                                                       
2020-11-21 Thiem D.    Djokovic N.                                   0.974286   
2020-11-22 Medvedev D. Thiem D.                                      0.932143   
           Thiem D.    Medvedev D.                                   0.962500   

                                    p2_lag_1_second_serve_cnt_match__median_28  \
date       p1          p2                                                        
2020-11-21 Thiem D.    Djokovic N.                                        22.0   
2020-11-22 Medvedev D. Thiem D.                                           31.0   
           Thiem D.    Medvedev D.                                        18.0   

                                    p2_filter_lag_1_max_points_in_a_row_match__pctl10_7  \
date       p1          p2                                                                 
2020-11-21 Thiem D.    Djokovic N.                                                3.6     
2020-11-22 Medvedev D. Thiem D.                                                   4.6     
           Thiem D.    Medvedev D.                                                5.2     

                                    p2_lag_1_first_serve_points_cnt_match__pctl90_28  \
date       p1          p2                                                              
2020-11-21 Thiem D.    Djokovic N.                                              45.3   
2020-11-22 Medvedev D. Thiem D.                                                 71.2   
           Thiem D.    Medvedev D.                                              48.6   

                                    p1_lag_1_receiver_points_won_match__pctl10_7  \
date       p1          p2                                                          
2020-11-21 Thiem D.    Djokovic N.                                          13.6   
2020-11-22 Medvedev D. Thiem D.                                             24.6   
           Thiem D.    Medvedev D.                                          13.6   

                                    p1_filter_lag_1_second_serve_points_cnt_match__median_7  \
date       p1          p2                                                                     
2020-11-21 Thiem D.    Djokovic N.                                               14.0         
2020-11-22 Medvedev D. Thiem D.                                                   9.0         
           Thiem D.    Medvedev D.                                               14.0         

                                    p1_lag_1_match_dur_pctl90_1  \
date       p1          p2                                         
2020-11-21 Thiem D.    Djokovic N.                         74.0   
2020-11-22 Medvedev D. Thiem D.                           155.0   
           Thiem D.    Medvedev D.                        174.0   

                                    p1_filter_lag_1_max_points_in_a_row_match__mean_7  \
date       p1          p2                                                               
2020-11-21 Thiem D.    Djokovic N.                                        

In [ ]:
## Парсер сделать так чтобы он заполнял ВСЕ поля которые нужны для дальнейших преобразований(как walkover????)

In [11]:
## IAN: 2021-01-01 - 2021-03-01 // EGOR: 2021-03-01 - 2021-05-21